# Compare vina1.1.2, vina1.2.7, qvina2.1

In [ ]:
# import libraries
import biotite.structure as struc
import biotite.structure.io.pdb as pdb
from biotite.structure import superimpose_homologs
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.MolStandardize import rdMolStandardize

from dockstring import load_target
import pandas as pd

from dockstring.utils import get_vina_filename

## Read aligned ligands

In [ ]:
# define complex-ligand dictionary
complex_ligand = {
'1S1C':'GNP',
'2ESM': 'M77',
'2ETK': 'HFS',
'2ETR': 'Y27',
'2V55': 'ANP',
'3D9V': 'H52',
'3NCZ': '3NC',
'3TV7': '07Q',
'3TWJ': '07R',
'3V8S': '0HD',
'4W7P': '3J7',
'4YVC': '4KH',
'4YVE': '4KK',
'5BML': '4TW',
'5HVU': '65R',
'5KKS': '6U1',
'5KKT': '6U2',
'5UZJ': '8UV',
'5WNE': 'B4J',
'5WNF': 'B4V',
'5WNG': 'B4Y',
'5WNH': 'B5G',
'6E9W': 'J0P',
'7JOU': 'VFS',
'7S25': '86G',
'7S26': '86K',
}

complex_ligand = dict(sorted(complex_ligand.items(), key=lambda item: item[1]))

molecules_order = []
for complex, ligand in complex_ligand.items():
    molecules_order.append(ligand)

print(molecules_order)


In [ ]:
# load target
target = load_target("ROCK1")

# load dataset and save ligands as smiles
dataset_path = "../materials/aligned_molecules.sdf"
with Chem.SDMolSupplier(dataset_path) as w:
    mols = [mol for mol in w]


ligands = {}
i = 0
for m in mols:
    ligands[molecules_order[i]] = Chem.MolToSmiles(m)
    i = i + 1

print(ligands)

In [ ]:
# helper function to neutralize docked molecules
def neutralize_mol(mol):
    for atom in mol.GetAtoms():
        if atom.GetFormalCharge() != 0:
            atom.SetFormalCharge(0)  
    return mol

In [ ]:
# read aligned poses
aligned_path = "../materials/aligned_molecules.sdf"
with Chem.SDMolSupplier(aligned_path) as w:
    al_mols = [mol for mol in w]

aligned = {}
i = 0
for m in al_mols:
    aligned[molecules_order[i]] = m
    i = i + 1

## Docking
* calculate RMS for each pose of each molecule

In [ ]:
scores = []
docked_mols = []
vina_filename = get_vina_filename()
print(vina_filename)
best_poses = {}
best_poses_confId = {}

for code, smiles in ligands.items():
    score, aux = target.dock(smiles)
    aff = aux["affinities"]
    #print(f"{code}: affinities = {aff}")
    docked = aux["ligand"]
    docked_mols.append(docked)
    rms_top = 100
    top_pose = docked
    top_confId = 0
    for x in range(0,len(aux["affinities"])):
        mol = Chem.Mol(docked, confId=x)
        score = aux["affinities"][x]
        al = neutralize_mol(aligned[code])
        doc = neutralize_mol(mol)
        rms = Chem.rdMolAlign.CalcRMS(doc, al) # reference in second argument

        scores.append({"code": code, "confId": x, "smiles": smiles, "score": score, "rms": rms})

        if(rms < rms_top):
            top_pose = mol
            top_confId = x
            rms_top = rms
    best_poses[code] = top_pose
    best_poses_confId[code] = top_confId
    print(f"{code}: best confId by RMS: {top_confId} with RMS = {rms_top}")


df = pd.DataFrame.from_records(scores)
print(df.head())


In [ ]:
# save outputs
# best by score
with Chem.SDWriter(f"../materials/docking/vina/{vina_filename}_docked.sdf") as wri:
    for mol in docked_mols:
        for conf_id in range(mol.GetNumConformers()): 
            #print(mol.GetNumConformers())
            mol.SetProp("_ConfID", str(conf_id)) 
            wri.write(mol, confId=conf_id) 
wri.close()

with Chem.SDWriter(f"../materials/graphs/{vina_filename}_docked.sdf") as wri:
    for mol in docked_mols:
        for conf_id in range(mol.GetNumConformers()): 
            #print(mol.GetNumConformers())
            mol.SetProp("_ConfID", str(conf_id)) 
            wri.write(mol, confId=conf_id) 
wri.close()